In [9]:
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import PolynomialFeatures

In this python program, we calculate the I-score for combination of the all various variable. 

In [3]:
#output the binary dataset

# data = pd.read_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\GMC_deal.csv").drop(columns='Unnamed: 0')
# orig_data = pd.read_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\gmc-training.csv").drop(columns='Unnamed: 0')

# data['x3'] = orig_data['NumberOfTime30-59DaysPastDueNotWorse']
# data['x8'] = orig_data['NumberOfTimes90DaysLate']
# data['x9'] = orig_data['NumberRealEstateLoansOrLines'] 
# data['x10'] = orig_data['NumberOfTime60-89DaysPastDueNotWorse']
# data['x11'] = orig_data['NumberOfDependents']

# #first step, I want to change this data into discrete type. So, I will using the guide provided by YC to deal with.
# #x3, x8, x9, x10, x11 are binary change, value = 0 if value==0 else 1.
# v1 = ['x3', 'x8', 'x9', 'x10', 'x11']
# for variable in v1 :
#     data[variable] = pd.np.where(data[variable]==0, 0, 1)

# #x1, x2, x4, x5, x7 are binary change value = 0 if value < median else 1.
# v2 = ['x1', 'x2', 'x4', 'x5', 'x7']
# for variable in v2 :
#     median = data[variable].median()
#     data[variable] = pd.np.where(data[variable]<=median, 0, 1)

# data.to_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\binary_GMC_deal.csv")

In [2]:
data = pd.read_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\GMC\\binary_GMC_deal.csv").drop(columns='Unnamed: 0')
X = data.drop(columns='y')
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
#set up I-score function 
def I_score(X, y, influential) :
    I = 0
    test = 0
    total_mean = np.mean(y)
    data = X.copy()
    data['y'] = y
    m = len(influential)
    values_set = itertools.product([0, 1], repeat=m)
    for values in values_set :
        table = data
        for value, variable in zip(values, influential) :
            table = table[table[variable]==value]
        group_amount = len(table)
        test  += group_amount
        if  group_amount :
            sub_mean = sum(table['y'])/group_amount
            I += (group_amount**2)*((sub_mean-total_mean)**2)
    I /= len(data)
    return I

#set up thin out function
def retaining(X, y, influential) :
    standard = I_score(X, y, influential)
    while True :
        thin_influential = 0
        for se_influential in itertools.combinations(influential, len(influential)-1) :
            I = I_score(X, y, se_influential)
            if I > standard : 
                thin_influential = se_influential
                standard = I
        if thin_influential == 0 : return influential, standard
        influential = thin_influential

In [14]:
#generate rank data

m_col = ['m=0', 'm=1', 'm=2', 'm=3', 'm=4', 'm=5', 'm=6']
value_col = ['I0', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6']
total_mean = y_train.mean()
total_table = pd.DataFrame()
for m, name, value in zip(range(1, 7), m_col, value_col) :
    name_list = []
    value_list = []
    for combination in itertools.combinations(X_train.columns, m) :
            I = I_score(X_train, y_train, combination)
            name_list.append(combination)
            value_list.append(I)
    temp_table = pd.DataFrame({m_col[m]:name_list, value_col[m]:value_list})
    temp_table = temp_table.sort_values(value_col[m], ascending=False)
    temp_table = temp_table.reset_index().drop(columns='index')
    total_table = pd.concat([total_table, temp_table], axis=1, ignore_index=True)
total_table.to_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\GMC\\I-score\\Iscore6.csv")

In [17]:
#thin out the variable

# for i in range(2, 7) :
#     result = []    
#     for influential, oI in zip(rank_data['m='+str(i)], rank_data['I'+str(i)]) :
#         d = dict()
#         if isinstance(influential, str) :
#             thin, I = retaining(X_train, y_train, eval(influential))
#             d['Originalindex'] = influential
#             d['OriginaliScore'] = oI
#             d['resulting'] = thin
#             d['ResultingiScore'] = I
#             result.append(d)
#         else : 
#             pd.DataFrame(result).sort_values("resulting").sort_values("ResultingiScore", ascending=False).to_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\GMC\\I-score\\Iscore"+str(i)+"_thin.csv")
#             break
#     pd.DataFrame(result).sort_values("resulting").sort_values("ResultingiScore", ascending=False).to_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\GMC\\I-score\\Iscore"+str(i)+"_thin.csv")

In [12]:
#make Logistic regression for retaining variables

for i in range(2, 7) :
    print('m=',i)
    s = set()
    thin_influential = pd.read_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\GMC\\I-score\\Iscore"+str(i)+"_thin.csv")['resulting']

    for thin in thin_influential :
        s.add(eval(thin))
    newpara_train = pd.DataFrame()
    newpara_test = pd.DataFrame()
    for thin in s :
        newpara_train[thin] = X_train[list(thin)].prod(axis=1)
        newpara_test[thin] = X_test[list(thin)].prod(axis=1)
    poly = PolynomialFeatures(3)
    newpara_train = poly.fit_transform(newpara_train)
    newpara_test = poly.fit_transform(newpara_test)
    model = LogisticRegression()
    model.fit(newpara_train, y_train)

    prob = model.predict_proba(newpara_train)[:, 1]
    auc = roc_auc_score(y_train, prob)
    print('train data AUC: %.3f' % auc)
    y_pred = model.predict(newpara_train)
    acc = accuracy_score(y_train, y_pred)
    print('train data Accuracy: %.3f' % acc)
    print('train data Error rate: %.3f' % (1-acc))
    precision = precision_score(y_train, y_pred)
    print('train data Precision: %.3f' % precision)
    recall = recall_score(y_train, y_pred)
    print('train data Recall: %.3f' % recall)
    f1 = f1_score(y_train, y_pred)
    print('train data F1 score: %.3f' % f1)

    prob = model.predict_proba(newpara_test)[:, 1]
    auc = roc_auc_score(y_test, prob)
    print('test data AUC: %.3f' % auc)
    y_pred = model.predict(newpara_test)
    acc = accuracy_score(y_test, y_pred)
    print('test data Accuracy: %.3f' % acc)
    print('test data Error rate: %.3f' % (1-acc))
    precision = precision_score(y_test, y_pred)
    print('test data Precision: %.3f' % precision)
    recall = recall_score(y_test, y_pred)
    print('test data Recall: %.3f' % recall)
    f1 = f1_score(y_test, y_pred)
    print('test data F1 score: %.3f' % f1)


m= 2


c:\Users\king0\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


train data AUC: 0.850
train data Accuracy: 0.936
train data Error rate: 0.064
train data Precision: 0.584
train data Recall: 0.155
train data F1 score: 0.245
test data AUC: 0.843
test data Accuracy: 0.938
test data Error rate: 0.062
test data Precision: 0.579
test data Recall: 0.168
test data F1 score: 0.261
m= 3


c:\Users\king0\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


train data AUC: 0.849
train data Accuracy: 0.935
train data Error rate: 0.065
train data Precision: 0.575
train data Recall: 0.151
train data F1 score: 0.240
test data AUC: 0.842
test data Accuracy: 0.938
test data Error rate: 0.062
test data Precision: 0.582
test data Recall: 0.169
test data F1 score: 0.262
m= 4


...

train data AUC: 0.847
train data Accuracy: 0.935
train data Error rate: 0.065
train data Precision: 0.572
train data Recall: 0.150
train data F1 score: 0.238
test data AUC: 0.841
test data Accuracy: 0.938
test data Error rate: 0.062
test data Precision: 0.585
test data Recall: 0.166
test data F1 score: 0.258
m= 5


c:\Users\king0\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


train data AUC: 0.847
train data Accuracy: 0.935
train data Error rate: 0.065
train data Precision: 0.573
train data Recall: 0.150
train data F1 score: 0.237
test data AUC: 0.841
test data Accuracy: 0.938
test data Error rate: 0.062
test data Precision: 0.574
test data Recall: 0.166
test data F1 score: 0.258
m= 6


c:\Users\king0\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


train data AUC: 0.840
train data Accuracy: 0.935
train data Error rate: 0.065
train data Precision: 0.566
train data Recall: 0.134
train data F1 score: 0.217
test data AUC: 0.835
test data Accuracy: 0.937
test data Error rate: 0.063
test data Precision: 0.570
test data Recall: 0.150
test data F1 score: 0.238


In [18]:
# m= 2
# train data AUC: 0.845
# train data Accuracy: 0.934
# train data Error rate: 0.066
# train data Precision: 0.546
# train data Recall: 0.152
# train data F1 score: 0.238
# test data AUC: 0.842
# test data Accuracy: 0.938
# test data Error rate: 0.062
# test data Precision: 0.571
# test data Recall: 0.171
# test data F1 score: 0.263
# m= 3
# train data AUC: 0.844
# train data Accuracy: 0.935
# train data Error rate: 0.065
# train data Precision: 0.559
# train data Recall: 0.147
# train data F1 score: 0.232
# test data AUC: 0.841
# test data Accuracy: 0.937
# test data Error rate: 0.063
# test data Precision: 0.571
# test data Recall: 0.165
# test data F1 score: 0.256
# m= 4
# train data AUC: 0.844
# train data Accuracy: 0.935
# train data Error rate: 0.065
# train data Precision: 0.558
# train data Recall: 0.150
# train data F1 score: 0.236
# test data AUC: 0.841
# test data Accuracy: 0.937
# test data Error rate: 0.063
# test data Precision: 0.558
# test data Recall: 0.166
# test data F1 score: 0.256
# m= 5
# train data AUC: 0.843
# train data Accuracy: 0.935
# train data Error rate: 0.065
# train data Precision: 0.560
# train data Recall: 0.147
# train data F1 score: 0.233
# test data AUC: 0.841
# test data Accuracy: 0.937
# test data Error rate: 0.063
# test data Precision: 0.568
# test data Recall: 0.165
# test data F1 score: 0.255
# m= 6
# train data AUC: 0.839
# train data Accuracy: 0.935
# train data Error rate: 0.065
# train data Precision: 0.562
# train data Recall: 0.127
# train data F1 score: 0.207
# test data AUC: 0.835
# test data Accuracy: 0.937
# test data Error rate: 0.063
# test data Precision: 0.576
# test data Recall: 0.144
# test data F1 score: 0.230

   col1 col2
2     1    a
1     2    b
0     3    c
